In [297]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *
from joblib import load, dump 
import pandas as pd
from decimal import Decimal

# Selenium Documentation: https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.remote.webelement

# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.add_argument("--headless")  
# chrome_options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'

In [312]:
driver = webdriver.Chrome('./chromedriver')
wait = WebDriverWait( driver, 10 )
URL = "https://www.tripadvisor.com/Hotels-g60763-New_York_City_New_York-Hotels.html"

In [313]:
driver.get(URL)
# click off screen so that the calendar does not get in the way 
time.sleep(2)
df = pd.DataFrame(columns=['Hotel_Name', 
                           'Price', 
                           'Primary_Rating', 
                           'Location_Rating', 
                           'Cleanliness_Rating', 
                           'Service_Rating', 
                           'Value_Rating',
                           'Num_Reviews',
                           'Address',
                           'Amenities'])
next_button = True

home_tab = driver.window_handles[0]

while(next_button != None):
    time.sleep(2)
    hotel_listings = driver.find_elements_by_class_name('respListingPhoto')
    time.sleep(2)
    for hotel in hotel_listings:
        time.sleep(2)
        try:
            print(hotel.get_attribute('href'))
            # stale element reference: https://stackoverflow.com/questions/44724185/element-myelement-is-not-clickable-at-point-x-y-other-element-would-receiv
            driver.execute_script("arguments[0].click();", hotel);
            tabs = driver.window_handles
            tabs.remove(home_tab)
            opened_tab = tabs[-1]
            driver.switch_to.window(opened_tab)
            time.sleep(2)
            df = df.append(scrape_hotel_data(driver), ignore_index=True)
            time.sleep(2)
            print('Rows: '  + str(len(df)))
            driver.close()
            driver.switch_to.window(home_tab)
        except StaleElementReferenceException:
            driver.refresh()
            break
            print('Error, skipping hotel listing')
    # driver.execute_script('window.scrollTo(0, document.body.scrollHeight*0.8);')
    time.sleep(2)
    # How to select multiple classes: http://makeseleniumeasy.com/2017/09/26/how-to-locate-web-element-which-has-multiple-class-names/
    next_button = driver.find_element_by_css_selector("a.nav.next")
    driver.execute_script("arguments[0].scrollIntoView()", next_button);
    driver.execute_script("arguments[0].click();", next_button);
    hotel_listings = []
    # next_button.click()        

dump(df, 'tripadvisor_dataframe.pkl', compress=True)
df.to_csv('tripadvisor_hotels.csv')

https://www.tripadvisor.com/Hotel_Review-g60763-d249711-Reviews-The_Hotel_at_Times_Square-New_York_City_New_York.html
68
4140
Rows: 1
https://www.tripadvisor.com/Hotel_Review-g60763-d93437-Reviews-Hotel_Edison-New_York_City_New_York.html
75
13283
Rows: 2
https://www.tripadvisor.com/Hotel_Review-g60763-d93579-Reviews-Park_Lane_Hotel-New_York_City_New_York.html
130
9391
Rows: 3
https://www.tripadvisor.com/Hotel_Review-g60763-d93454-Reviews-Crowne_Plaza_Times_Square_Manhattan-New_York_City_New_York.html
89
11937
Rows: 4
https://www.tripadvisor.com/Hotel_Review-g60763-d122005-Reviews-The_New_Yorker_a_Wyndham_Hotel-New_York_City_New_York.html
80
14918
Rows: 5
https://www.tripadvisor.com/Hotel_Review-g60763-d1938661-Reviews-Row_NYC_Hotel-New_York_City_New_York.html
72
25475
Rows: 6
https://www.tripadvisor.com/Hotel_Review-g60763-d99388-Reviews-The_Hotel_New_York_City-New_York_City_New_York.html
64
1835
Rows: 7
https://www.tripadvisor.com/Hotel_Review-g60763-d93520-Reviews-Park_Central_Hotel_

149
2932
Rows: 61
https://www.tripadvisor.com/Hotel_Review-g60763-d93390-Reviews-Hotel_Belleclaire-New_York_City_New_York.html
109
4344
Rows: 62
https://www.tripadvisor.com/Hotel_Review-g60763-d93612-Reviews-Shoreham-New_York_City_New_York.html
94
2501
Rows: 63
https://www.tripadvisor.com/Hotel_Review-g60763-d514488-Reviews-Hotel_on_Rivington-New_York_City_New_York.html
140
1861
Rows: 64
https://www.tripadvisor.com/Hotel_Review-g60763-d12932808-Reviews-AC_Hotel_New_York_Downtown-New_York_City_New_York.html
115
149
Rows: 65
https://www.tripadvisor.com/Hotel_Review-g60763-d93589-Reviews-Michelangelo_Hotel-New_York_City_New_York.html
200
4546
Rows: 66
https://www.tripadvisor.com/Hotel_Review-g60763-d1595302-Reviews-Club_Quarters_Hotel_World_Trade_Center-New_York_City_New_York.html
104
2341
Rows: 67
https://www.tripadvisor.com/Hotel_Review-g60763-d93543-Reviews-Shelburne_Hotel_Suites_by_Affinia-New_York_City_New_York.html
113
5572
Rows: 68
https://www.tripadvisor.com/Hotel_Review-g60763-d1

95
2010
Rows: 120
https://www.tripadvisor.com/Hotel_Review-g60763-d112066-Reviews-The_Maxwell_New_York_City-New_York_City_New_York.html
103
5735
Rows: 121
https://www.tripadvisor.com/Hotel_Review-g60763-d93376-Reviews-OYO_Times_Square-New_York_City_New_York.html
73
5435
Rows: 122
https://www.tripadvisor.com/Hotel_Review-g60763-d2554351-Reviews-Hyatt_Place_New_York_Midtown_South-New_York_City_New_York.html
125
2867
Rows: 123
https://www.tripadvisor.com/Hotel_Review-g60763-d93566-Reviews-The_Surrey-New_York_City_New_York.html
560
2362
Rows: 124
https://www.tripadvisor.com/Hotel_Review-g60763-d12845266-Reviews-Mondrian_Park_Avenue-New_York_City_New_York.html
145
656
Rows: 125
https://www.tripadvisor.com/Hotel_Review-g60763-d93382-Reviews-Washington_Square_Hotel-New_York_City_New_York.html
184
1850
Rows: 126
https://www.tripadvisor.com/Hotel_Review-g60763-d93351-Reviews-The_Frederick_Hotel-New_York_City_New_York.html
114
2524
Rows: 127
https://www.tripadvisor.com/Hotel_Review-g60763-d99387

127
154
Rows: 179
https://www.tripadvisor.com/Hotel_Review-g60763-d564851-Reviews-Courtyard_New_York_Manhattan_Upper_East_Side-New_York_City_New_York.html
89
1073
Rows: 180
https://www.tripadvisor.com/Hotel_Review-g60763-d1155229-Reviews-Royal_Park_Hotel-New_York_City_New_York.html
66
765
Rows: 181
https://www.tripadvisor.com/Hotel_Review-g60763-d122014-Reviews-Gild_Hall_A_Thompson_Hotel-New_York_City_New_York.html
154
2325
Rows: 182
https://www.tripadvisor.com/Hotel_Review-g60763-d80107-Reviews-Roger_Smith_Hotel-New_York_City_New_York.html
110
2710
Rows: 183
https://www.tripadvisor.com/Hotel_Review-g60763-d5960548-Reviews-Hilton_Garden_Inn_New_York_Manhattan_Midtown_East-New_York_City_New_York.html
110
1697
Rows: 184
https://www.tripadvisor.com/Hotel_Review-g60763-d866615-Reviews-Soho_54-New_York_City_New_York.html
71
1960
Rows: 185
https://www.tripadvisor.com/Hotel_Review-g60763-d224223-Reviews-Nyma_the_New_York_Manhattan_Hotel-New_York_City_New_York.html
69
3139
Rows: 186
https://ww

89
243
Rows: 238
https://www.tripadvisor.com/Hotel_Review-g60763-d14920907-Reviews-Mr_C_Seaport-New_York_City_New_York.html
169
155
Rows: 239
https://www.tripadvisor.com/Hotel_Review-g60763-d99368-Reviews-Courtyard_New_York_Manhattan_Times_Square-New_York_City_New_York.html
143
1639
Rows: 240
https://www.tripadvisor.com/Hotel_Review-g60763-d290987-Reviews-St_Marks_Hotel-New_York_City_New_York.html
102
518
Rows: 241
https://www.tripadvisor.com/Hotel_Review-g60763-d1415127-Reviews-Hotel_Scherman-New_York_City_New_York.html
107
1372
Rows: 242
https://www.tripadvisor.com/Hotel_Review-g60763-d1308397-Reviews-West_57th_Street_by_Hilton_Club-New_York_City_New_York.html
305
1776
Rows: 243
https://www.tripadvisor.com/Hotel_Review-g60763-d3533197-Reviews-Hyatt_Union_Square_New_York-New_York_City_New_York.html
193
2096
Rows: 244
https://www.tripadvisor.com/Hotel_Review-g60763-d1791248-Reviews-Hotel_Mulberry-New_York_City_New_York.html
109
696
Rows: 245
https://www.tripadvisor.com/Hotel_Review-g60

...

0
5
Rows: 550
https://www.tripadvisor.com/Hotel_Review-g60763-d320562-Reviews-YourStay_MidTown_Apartments-New_York_City_New_York.html
0
52
Rows: 551
https://www.tripadvisor.com/Hotel_Review-g60763-d1452077-Reviews-Margot_Guest_House-New_York_City_New_York.html
0
73
Rows: 552
https://www.tripadvisor.com/Hotel_Review-g60763-d15632114-Reviews-Excellent_Apartment_by_Central_Park-New_York_City_New_York.html
0
Rows: 553
https://www.tripadvisor.com/Hotel_Review-g60763-d19917464-Reviews-House_Zaza-New_York_City_New_York.html
0
Rows: 554
https://www.tripadvisor.com/Hotel_Review-g60763-d1811422-Reviews-Lincoln_Road_Bed_Breakfast-New_York_City_New_York.html
0
4
Rows: 555
https://www.tripadvisor.com/Hotel_Review-g60763-d1605446-Reviews-Apt_168-New_York_City_New_York.html
0
20
Rows: 556
https://www.tripadvisor.com/Hotel_Review-g60763-d93418-Reviews-Bristol_Plaza-New_York_City_New_York.html
0
15
Rows: 557
https://www.tripadvisor.com/Hotel_Review-g60763-d11737011-Reviews-Shepherd_Guest_House-New_York

0
130
Rows: 615
https://www.tripadvisor.com/Hotel_Review-g60763-d668187-Reviews-1600_Broadway_on_the_Square-New_York_City_New_York.html
0
19
Rows: 616
https://www.tripadvisor.com/Hotel_Review-g60763-d17018172-Reviews-116_John_Street_Apartments-New_York_City_New_York.html
0
2
Rows: 617
https://www.tripadvisor.com/Hotel_Review-g60763-d1034304-Reviews-Michael-New_York_City_New_York.html
0
40
Rows: 618
https://www.tripadvisor.com/Hotel_Review-g60763-d17950243-Reviews-Bei_NY_Queens-New_York_City_New_York.html
0
Rows: 619
https://www.tripadvisor.com/Hotel_Review-g60763-d15688556-Reviews-Bluebird_Suites_in_Times_Square-New_York_City_New_York.html
0
Rows: 620
https://www.tripadvisor.com/Hotel_Review-g60763-d15637691-Reviews-The_Modern_Rockefeller-New_York_City_New_York.html
0
1
Rows: 621
https://www.tripadvisor.com/Hotel_Review-g60763-d15688551-Reviews-Bluebird_Suites_in_Midtown_West-New_York_City_New_York.html
0
Rows: 622
https://www.tripadvisor.com/Hotel_Review-g60763-d7906966-Reviews-The_Me

0
32
Rows: 678
https://www.tripadvisor.com/Hotel_Review-g60763-d12224088-Reviews-Chelsea_Art_Suites-New_York_City_New_York.html
0
1
Rows: 679
https://www.tripadvisor.com/Hotel_Review-g60763-d93489-Reviews-Murray_Hill_East_Suites-New_York_City_New_York.html
0
677
Rows: 680
https://www.tripadvisor.com/Hotel_Review-g60763-d16750237-Reviews-Global_Luxury_Suites_at_300_East_39th_Street-New_York_City_New_York.html
0
Rows: 681
https://www.tripadvisor.com/Hotel_Review-g60763-d12722932-Reviews-Macy_Empire-New_York_City_New_York.html
0
4
Rows: 682
https://www.tripadvisor.com/Hotel_Review-g60763-d649058-Reviews-Hotel_Alexander-New_York_City_New_York.html
0
356
Rows: 683
https://www.tripadvisor.com/Hotel_Review-g60763-d620040-Reviews-Wingate_by_Wyndham_Manhattan_Midtown-New_York_City_New_York.html
0
2437
Rows: 684
https://www.tripadvisor.com/Hotel_Review-g60763-d15697497-Reviews-Studio_Apartments-New_York_City_New_York.html
0
1
Rows: 685
https://www.tripadvisor.com/Hotel_Review-g60763-d5483129-Rev

0
189
Rows: 741
https://www.tripadvisor.com/Hotel_Review-g60763-d17637656-Reviews-The_East_Village_Home-New_York_City_New_York.html
0
Rows: 742
https://www.tripadvisor.com/Hotel_Review-g60763-d1752366-Reviews-Prive-New_York_City_New_York.html
0
46
Rows: 743
https://www.tripadvisor.com/Hotel_Review-g60763-d15737063-Reviews-Hamilton_Heights_Lovely_Apartments-New_York_City_New_York.html
0
Rows: 744
https://www.tripadvisor.com/Hotel_Review-g60763-d1546341-Reviews-Hotel_99-New_York_City_New_York.html
0
157
Rows: 745
https://www.tripadvisor.com/Hotel_Review-g60763-d260396-Reviews-The_Amsterdam_Inn-New_York_City_New_York.html
0
111
Rows: 746
https://www.tripadvisor.com/Hotel_Review-g60763-d208888-Reviews-Riff_Chelsea-New_York_City_New_York.html
0
276
Rows: 747
https://www.tripadvisor.com/Hotel_Review-g60763-d665814-Reviews-The_Soho_Loft-New_York_City_New_York.html
0
26
Rows: 748
https://www.tripadvisor.com/Hotel_Review-g60763-d259717-Reviews-Efuru_Guest_House-New_York_City_New_York.html
0
77


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.nav.next"}
  (Session info: chrome=80.0.3987.132)


In [310]:
def scrape_hotel_data(driver):
    page = driver.page_source
    soup_expansion = BeautifulSoup(page, 'html.parser')
    address = soup_expansion.find('span', class_='public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI')
    hotel_address = address.get_text() if address != None else 'Not Found'
    hotel_name = driver.title.split('$')[0].title()
    price_tag = soup_expansion.find('span', class_='hotels-hotel-offers-DetailTextOffer__price--2gYdd')
    price = 0
    if price_tag != None:
        try:
            price = int(re.sub(r'[^\d.]', '', re.search('\$(\d+)', driver.title).group(0)))
        except AttributeError:
            try:
                price = int(re.sub(r'[^\d.]', '', re.search('\$(\d+)', price_tag.get_text()).group(0)))
            except:
                price = 0
    print(price)
    num_reviews_tag = soup_expansion.find('span', class_='hotels-hotel-review-about-with-photos-Reviews__seeAllReviews--3PpLR')
    num_reviews = 0
    if num_reviews_tag != None:
        try:
            num_reviews_string = num_reviews_tag.get_text()
            num_reviews = int(num_reviews_string.split(' ')[0].replace(',', ''))
            print(num_reviews)
        except:
            print('unable to get num_reviews: ', num_reviews_string)
    primary_rating_tag = soup_expansion.find('span', class_='hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA')
    primary_rating = Decimal(primary_rating_tag.get_text()) if primary_rating_tag != None else 0
    ui_bubble_ratings = {}
    for i in range(0, 55, 5):
        ui_bubble_ratings['bubble_' + str(i)] = i * 0.1
    secondary_ratings = soup_expansion.find_all('div', class_='hotels-hotel-review-about-with-photos-Reviews__subratingRow--2u0CJ')
    metrics = [None, None, None, None]
    for i, rating in enumerate(secondary_ratings):
        result = re.search('bubble_(\d\d)', str(rating))
        metrics[i] = Decimal(ui_bubble_ratings[result.group(0)])

    amenities_tag = soup_expansion.find_all('div', class_='hotels-hr-about-amenities-Amenity__amenity--3fbBj')
    amenities = []
    for amenity in amenities_tag:
        amenities.append(amenity.get_text())
    return {'Hotel_Name': hotel_name, 
            'Price': price, 
            'Primary_Rating': primary_rating, 
            'Location_Rating': metrics[0], 
            'Cleanliness_Rating': metrics[1], 
            'Service_Rating': metrics[2], 
            'Value_Rating': metrics[3], 
            'Num_Reviews': num_reviews,
            'Address': hotel_address,
            'Amenities': amenities}

In [314]:
dump(df, 'tripadvisor_dataframe.pkl', compress=True)
df.to_csv('tripadvisor_hotels.csv')
len(df)

755

In [316]:
df.tail()

,Hotel_Name,Price,Primary_Rating,Location_Rating,Cleanliness_Rating,Service_Rating,Value_Rating,Num_Reviews,Address,Amenities
750,Residences At 51St Street And 7Th Avenue - Con...,0,4.0,None,None,None,None,2,"150 W 51st St, New York City, NY 10019-6836",[]
751,Hotel Richland New York - Reviews (New York Ci...,0,4.0,4,4.5,4.5,4.5,706,"5 Allen St, New York City, NY 10002-5301","[Free High Speed Internet (WiFi), Free interne..."
752,Loft 109 Ny - Prices & Lodging Reviews (New Yo...,0,2.0,None,None,None,None,1,"247 W 109th St, New York City, NY 10025-2372",[]
753,Apartments Upper West Side - Prices & Condomin...,0,0,None,None,None,None,0,"West 97th Street and Columbus Avenue, New York...",[]
754,Midtown West Suites At 48Th Street And Times S...,0,0,None,None,None,None,0,"235 W 48th St, New York City, NY 10036-1404",[]


In [189]:
# driver.get(URL)
# hotel_listings = driver.find_elements_by_class_name('respListingPhoto')
# hotel_listings[0].click()
# tested_hotel_page = driver.current_url

hotel_page = 'https://www.tripadvisor.com/Hotel_Review-g60763-d121994-Reviews-or825-HI_New_York_City_Hostel-New_York_City_New_York.html'
driver.get(hotel_page)
page = driver.page_source
soup_expansion = BeautifulSoup(page, 'html.parser')
hotel_name_title = driver.title

In [190]:
print(hotel_name_title)
hotel_name = ''
for name in hotel_name_title.split(' '):
    if '$' not in name:
        hotel_name += name + ' '
    else:
        price = name[1:]
        break
print(price)
print(hotel_name.title())

HI NEW YORK CITY HOSTEL $30 ($̶5̶5̶) - Prices & Reviews - Tripadvisor
30
Hi New York City Hostel 


In [191]:
primary_rating = soup_expansion.find('span', class_='hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA')
primary_rating.get_text()

'4.0'

In [192]:
import re
ui_bubble_ratings = {}
for i in range(0, 50, 5):
    ui_bubble_ratings['bubble_' + str(i)] = i * 0.1

secondary_ratings = soup_expansion.find_all('div', class_='hotels-hotel-review-about-with-photos-Reviews__subratingRow--2u0CJ')
for rating in secondary_ratings:
    metric = rating.get_text()
    result = re.search('bubble_(\d\d)', str(rating))
    print(metric + ': ' + str(ui_bubble_ratings[result.group(0)]))
    
amenities = soup_expansion.find_all('div', class_='hotels-hr-about-amenities-Amenity__amenity--3fbBj')
for amenity in amenities:
    print(amenity.get_text())

Location: 4.0
Cleanliness: 4.0
Service: 4.0
Value: 4.0
Free High Speed Internet (WiFi)
Coffee shop
Game room
Billiards
BBQ facilities
Baggage storage
24-hour front desk
Express check-in / check-out
Free internet
Wifi
Public wifi
Internet
Restaurant
Snack bar
Vending machine
Entertainment staff
Evening entertainment
Table tennis
Convenience store
Non-smoking hotel
Outdoor furniture
Picnic area
Shared bathroom
Shared kitchen
Shared lounge / TV area
Sun terrace
Laundry service
Self-serve laundry


In [123]:
page_title = 'HOTEL PENNSYLVANIA $56 ($̶2̶0̶2̶) - Updated 2020 Prices & Reviews - New York City - Tripadvisor'
print(page_title.split('$')[0].title())
print(re.search('\$(\d+)', page_title).group(0))

Hotel Pennsylvania 
$56


In [193]:
address = soup_expansion.find('span', class_='public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI')
address.get_text()

'891 Amsterdam Ave, New York City, NY 10025-4403'